In [16]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
#
from langchain_community.llms import HuggingFaceHub
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate
#
from langchain.chains import RetrievalQA
#
import  numpy as np
#
import os
from dotenv import load_dotenv

In [18]:
load_dotenv()

hf_token = os.getenv('HF_TOKEN')
os.environ['HF_TOKEN'] = hf_token
os.environ['HUGGINGFACEHUB_API_TOKEN'] = hf_token

In [5]:
loader = PyPDFDirectoryLoader("./koldogovor")

documents = loader.load()
len(documents)

49

In [7]:
documents[19]

Document(page_content=' \n149 \n  \n2. Направления и размер оказания материальной помощи  \nработникам ТПУ  \n \n2.1. Материальная помощь на частичное возмещение затрат в связи  \nс дорогостоящим лечением оказывается  работникам университета, \nсреднемесячная заработная плата которых не превышает средней \nзаработной платы по ТПУ за предыдущий год, при предоставлении ими \nподтверждающих необходимость такого лечения документов:  \n- на приоб ретение лекарств и иных лечебных препаратов, оборудования \nпо жизненно важным показаниям  (наличие инвалидности, перенесенное \nзаболевание, повлекшее необходимость постоянного или длительного \nприема лекарств) - не более 3 тысяч рублей. Подтверждающими \nдокумен тами являются справка об инвалидности, выписка из \nмедицинского учреждения, назначения от врача, рецепты, кассовые и \nтоварные чеки;  \n- на лечение и протезирование зубов – не более 3 тысяч рублей. \nПодтверждающими документами являются договор на оказание услуг,  \nакт выполненных ра

In [10]:
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

final_documents = text_splitter.split_documents(documents)
len(final_documents)

260

In [11]:
final_documents[19]

Document(page_content='78 \n \uf0b7 Почетный член Томского политехнического университета;  \n\uf0b7 Почетный выпускник Томского политехнического университета;  \n\uf0b7 Профессор практики Томского политехнического университета.  \n1.3. Представление к награждению знаками отличия ТПУ работников ТПУ \nосуществляется при соблюдении следующей  последовательности \nнаграждения: Благодарственное письмо ректора ТПУ - Благодарность ТПУ – \nПочетная грамота ТПУ - Медаль «За участие в развитии Томского', metadata={'source': 'koldogovor\\koldogovor06.pdf', 'page': 1})

In [12]:
#model_id = 'sentence-transformers/all-MiniLM-l6-v2'
model_id = 'BAAI/bge-small-en-v1.5'

#model_args = {'device': 'cuda'}
model_args = {'device': 'cpu'}

huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_id,
    model_kwargs=model_args,
    encode_kwargs={'normalize_embeddings': True}
)

e:\Progi\PythonProg\NLP\LangChain\KrishNaik\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [14]:
#np.array(huggingface_embeddings.embed_query(final_documents[19].page_content))

In [15]:
np.array(huggingface_embeddings.embed_query(final_documents[19].page_content)).shape

(384,)

In [17]:
vector_store = FAISS.from_documents(final_documents, huggingface_embeddings)

In [19]:
query = 'Почетные звания Томского политехнического университета?'

In [20]:
relevant_docments = vector_store.similarity_search(query)
print( len(relevant_docments) )
relevant_docments[0].page_content

4


'установление дополнительных выплат стимулирующего характера в \nсоответствии с Положение м об оплате труда в Томском политехническом \nуниверситет е. \n \n12. Порядок лишения наград и почетного звания'

In [21]:
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E844047740>, search_kwargs={'k': 3})

In [22]:
hf = HuggingFaceHub(
    #repo_id='mistralai/Mistral-7B-v0.1',
    repo_id='RefalMachine/ruadapt_qwen2.5_3B_ext_u48_instruct_v4',
    model_kwargs={'temperature': 0.1, 'max_length': 500}
)

hf.invoke(query)

e:\Progi\PythonProg\NLP\LangChain\KrishNaik\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


'Почетные звания Томского политехнического университета? Какие есть и какие получить?\nВ Томском политехническом университете (ТПУ) существует система награждения сотрудников и студентов за выдающиеся достижения, научные работы, инновации и вклад в развитие вуза. Вот некоторые из наиболее значимых и престижных наград, которые могут быть получены:\n\n1. **Орден Трудового Красного Знамени** - высшая государственная награда Российской Федерации, присуждаемая за выдающиеся заслуги в труде.\n\n2. **Орден Ленина** - одна из высших государственных наград СССР и России, вручаемая'

In [2]:
# Взято с https://huggingface.co/RefalMachine/ruadapt_qwen2.5_3B_ext_u48_instruct_v4
hf = HuggingFacePipeline.from_model_id(
    #model_id='mistralai/Mistral-7B-v0.1',
    model_id='RefalMachine/ruadapt_qwen2.5_3B_ext_u48_instruct_v4',
    task='text-generation',
    pipeline_kwargs={'temperature': 0, 'max_new_tokens': 300}
)

llm = hf 
llm.invoke(query)

NameError: name 'HuggingFacePipeline' is not defined